In [1]:
from contracting.stdlib.bridge.hashing import sha3
import secrets

In [2]:
def rps_contract():
    # This tells the blockchain to make space for data. Variable and Hash are not Python builtins. 
    # They are globals (globals are something you can access without importing it) made available by the contracting. 
    # Calling Variable() creates a new piece of memory within the blockchain, that can hold a single value. 
    # If we say "foo = Variable()" then we can interact with that piece of memory through "foo". 
    # We can set the piece of memory to 1, for example by calling "foo.set(1)".
    # Setting the value will change the value for the next call to the smart contract.
    # We can get the piece of memory by calling "foo.get()".
    next_game_id = Variable()
    
    game_id_to_password_hash = Hash()
    game_id_to_player1_choice = Hash()
            
    # @construct means that this is the function that will be called ONCE when the smart contract is created
    @construct
    def constructor():
        # Game id starts at 0.
        next_game_id.set(0)
        
    # @export makes this function public, so it can be called by anyone on a deployed contract    
    @export
    def start_game(password_hash, choice_hash):
        # This retrieves unique game Id from the blockchain. 
        unique_game_id = next_game_id.get()
        # This increments the number by 1, so the next game has a unique Id.
        next_game_id.set(next_game_id.get() + 1)
        
        game_id_to_password_hash[unique_game_id] = password_hash
        
from contracting.client import ContractingClient
client = ContractingClient(signer='ren')

client.submit(rps_contract)

contract = client.get_contract('rps_contract')


Alice (player 1) chooses single use password. Only the person that has the password can join the game and play with Alice. Everything that starts with alice_ is only visible to Alice. 

In [3]:
alice_game_password = "trollbridge"

Alice hashes the password so she can submit it to the blockchain without sharing the actual password. She does this because everything on the blockchain is public, and she wants only the person she chooses to play the game with to have the password. 

*explain hashing

In [4]:
alice_game_password_hash = sha3(alice_game_password)
alice_game_password_hash

'47a5bcfb0d81053f5025ab57e6b94f43751f91bdb16fc0d63595223dc78ec1b4'

Alice chooses rock.

In [5]:
alice_choice = "rock"

We can't submit the choice to the blockchain as plain text, because then Bob (player 2) can see it and win by choosing paper. 

*explain blockchain sequentiality

In [6]:
alice_choice_hash = sha3(alice_choice)
alice_choice_hash

'bd996e2dc82a7c3e2c2da41291648e852b62b01fb09bcb6ed77975c273f08404'

The problem with subiting Alices choice like this is the 3 choices will be hashed the same everytime. Bob (player 2) can know what each of the hashes for the 3 choices and pick paper to win. To fix this Alice needs to pick a random salt to hash with her choice so that Bob can't guess her choice by looking at the hash.

In [7]:
alice_choice_salt = secrets.token_hex(32)
alice_choice_salt

'804b82ac8c9fa1b5843dbdd614bfdaac1f2b171acc33b87ce0fb2e78321eb748'

In [8]:
alice_salted_choice = alice_choice + alice_choice_salt
alice_salted_choice

'rock804b82ac8c9fa1b5843dbdd614bfdaac1f2b171acc33b87ce0fb2e78321eb748'

In [9]:
alice_salted_choice_hash = sha3(alice_salted_choice)
alice_salted_choice_hash

'ad4ef7f8adb2b1dd2608b61147094ffe99ade7624edb48d8975e9f8117211cc0'

Now Alice starts a game so she can invite Bob to play.

In [10]:
contract.start_game(password_hash=alice_game_password_hash, choice_hash=alice_salted_choice_hash)

object of type 'int' has no len()


TypeError: object of type 'int' has no len()